In [1]:
from PIL import Image
from pathlib import Path
import time
import cv2
import mss
import numpy as np
import imutils

In [2]:
def read(p):
    return cv2.imread(p)
def show(cv_img):
    cv2.imshow("test", cv_img)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [14]:
img_path = r'..\data\two_hands.png'
im = cv2.imread(img_path)
gray = cv2.cvtColor(im, cv2.COLOR_BGRA2GRAY)
show(im)

In [15]:
_, thres = cv2.threshold(gray, 254, 255, cv2.THRESH_BINARY)
show(thres)

In [16]:
def sort_contours(cnts):
	i = 0
	boundingBoxes = [cv2.boundingRect(c) for c in cnts]
	(cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
		key=lambda b:b[1][i]))
 
	# return the list of sorted contours and bounding boxes
	return (cnts, boundingBoxes)

def draw_contour(image, c, i):
	M = cv2.moments(c)
	cX = int(M["m10"] / M["m00"])
	cY = int(M["m01"] / M["m00"])
 
	cv2.putText(image, "#{}".format(i + 1), (cX, cY), cv2.FONT_HERSHEY_SIMPLEX,
		0.5, (0, 255, 0), 1)
 
	return image

def get_key_img(thres_img, cnt):
	mask = np.zeros(thres_img.shape, dtype='uint8')
	cv2.drawContours(mask, [cnt], -1, 255, -1)
	res = cv2.bitwise_and(thres_img, thres_img, mask=mask)
	return res

def get_key_roi(thres_img, boxes):
	res = thres_img[boxes[1]:(boxes[1]+boxes[3]), boxes[0]:(boxes[0]+boxes[2])]
	return res

In [17]:
cnts = cv2.findContours(thres.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
orig = im.copy()

cv2.drawContours(orig, cnts, -1, (255, 0, 0), 1)
show(orig)

(cnts, boundingBoxes) = sort_contours(cnts)

sorted_cnts = im.copy()
for (i, c) in enumerate(cnts):
	draw_contour(sorted_cnts, c, i)

show(sorted_cnts)

In [18]:
key0_img = get_key_img(thres, cnts[0])
show(key0_img)

In [26]:
key0_roi = get_key_roi(im, boundingBoxes[0])
show(key0_roi)

In [42]:
key_roi = get_key_roi(im, boundingBoxes[0])
show(key_roi)
b, g, r = cv2.split(key_roi)
np.sum(b), np.sum(r)

(143393, 92413)

In [43]:
key_roi = get_key_roi(im, boundingBoxes[1])
show(key_roi)
b, g, r = cv2.split(key_roi)
np.sum(b), np.sum(r)


(148445, 129517)

In [27]:
key_roi = get_key_roi(im, boundingBoxes[0])
hsv = cv2.cvtColor(key_roi, cv2.COLOR_BGR2HSV)

lower_blue = np.array([110,100,100])
upper_blue = np.array([130,255,255])
mask = cv2.inRange(hsv, lower_blue, upper_blue)
show(mask)
cnt_blue = cv2.countNonZero(mask)

lower_red = np.array([-10,100,100])
upper_red = np.array([10,255,255])
mask = cv2.inRange(hsv, lower_red, upper_red)
show(mask)
cnt_red = cv2.countNonZero(mask)

cnt_blue, cnt_red

(231, 0)

In [11]:
class KeysDetector:
    UP = "UP"
    DOWN = "DOWN"
    LEFT = "LEFT"
    RIGHT = "RIGHT"
    UP_LEFT = "UP_LEFT"
    DOWN_LEFT = "DOWN_LEFT"
    UP_RIGHT = "UP_RIGHT"
    DOWN_RIGHT = "DOWN_RIGHT"

def get_dir(roi):
    h, w = roi.shape
    reg0 = roi[:, 0 : w // 3]
    reg1 = roi[0 : h // 3, :]
    reg2 = roi[:, 2 * w // 3 : w]
    reg3 = roi[2 * h // 3 : h, :]

    rate0 = cv2.countNonZero(reg0)
    rate1 = cv2.countNonZero(reg1)
    rate2 = cv2.countNonZero(reg2)
    rate3 = cv2.countNonZero(reg3)

    arr = np.array((rate0, rate1, rate2, rate3))
    print(arr)
    sorted_idx = arr.argsort()[::-1]
    print(sorted_idx)
    max_idx1 = sorted_idx[0]
    max_idx2 = sorted_idx[1]
    max1 = arr[max_idx1]
    max2 = arr[max_idx2]
    if max2 / max1 < 0.63:
        if max_idx1 == 0: return KeysDetector.LEFT
        elif max_idx1 == 1: return KeysDetector.UP
        elif max_idx1 == 2: return KeysDetector.RIGHT
        elif max_idx1 == 3: return KeysDetector.DOWN
    else:
        if max_idx1 == 0 and max_idx2 == 1: return KeysDetector.UP_LEFT
        elif max_idx2 == 0 and max_idx1 == 1: return KeysDetector.UP_LEFT

        elif max_idx1 == 0 and max_idx2 == 3: return KeysDetector.DOWN_LEFT
        elif max_idx2 == 0 and max_idx1 == 3: return KeysDetector.DOWN_LEFT
        
        elif max_idx1 == 2 and max_idx2 == 1: return KeysDetector.UP_RIGHT
        elif max_idx2 == 2 and max_idx1 == 1: return KeysDetector.UP_RIGHT

        elif max_idx1 == 2 and max_idx2 == 3: return KeysDetector.DOWN_RIGHT
        elif max_idx2 == 2 and max_idx1 == 3: return KeysDetector.DOWN_RIGHT

        else: return KeysDetector.UP

In [12]:
for i in range(len(boundingBoxes)):
    key_roi = get_key_roi(thres, boundingBoxes[i])
    direction = get_dir(key_roi)
    print(direction)

[69 45 50 89]
[3 0 2 1]
DOWN_LEFT
[41 96 99 51]
[2 1 3 0]
UP_RIGHT
[30 80 40 34]
[1 2 3 0]
UP
[70 92 53 47]
[1 0 2 3]
UP_LEFT
[69 45 50 89]
[3 0 2 1]
DOWN_LEFT
[32 31 39 90]
[3 2 0 1]
DOWN
[40 32 38 95]
[3 0 2 1]
DOWN
[36 34 85 40]
[2 3 0 1]
RIGHT
[36 75 88 50]
[2 1 3 0]
UP_RIGHT
[69 46 50 90]
[3 0 2 1]
DOWN_LEFT
[39 31 36 92]
[3 0 2 1]
DOWN


In [58]:
show(thres)

In [59]:
thres.shape

(40, 370)

In [60]:
thres[:,0:10].shape

(40, 10)